# Initialization

In [1]:
import os
import cv2
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util

%matplotlib inline

In [3]:
paths = {
    'TENSORFLOW': os.path.join('Tensorflow'),
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'EXPORTED_MODEL_PATH': os.path.join('Tensorflow', 'workspace', 'exported-model'),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'PROTOC_PATH':os.path.join('Tensorflow','protoc')
}
files = {
    'LABEL_MAP': os.path.join(paths['ANNOTATION_PATH'], "label_map.pbtxt"),
    "EXPORTED_MODEL_PIPELINE": os.path.join(paths["EXPORTED_MODEL_PATH"], "pipeline.config")
}
# labels = ["phone", "pistol", "hand"]
labels = ["pistol"]
for path in paths.values():
  if not os.path.exists(path):
    !mkdir {path}

# label map

In [3]:
template = '''item {
    id: $
    name: '%'
}
'''
label_map = []
for i in range(len(labels)):
  label_map.append(template)
  label_map[i] = label_map[i].replace("$", str(i + 1))
  label_map[i] = label_map[i].replace("%", labels[i])
print("\n".join(label_map))

with open(os.path.join(paths["ANNOTATION_PATH"], "label_map.pbtxt"), "w") as f:
  f.write("\n".join(label_map))

item {
    id: 1
    name: 'pistol'
}



# importing model

In [4]:
model = tf.saved_model.load(os.path.join(paths["EXPORTED_MODEL_PATH"], "saved_model"))
category_index = label_map_util.create_category_index_from_labelmap(files['LABEL_MAP'])
detect_fn = model.signatures['serving_default']


# Detection

--------------------------------

In [18]:
# IMAGE_PATH = random.choice(test_images)
IMAGE_PATH = r"C:\Users\USER\Desktop\Cyber\PRJ\1.jpg"
print(IMAGE_PATH)
img = cv2.cvtColor(cv2.imread(IMAGE_PATH), cv2.COLOR_BGR2RGB)
image_np = np.array(img)
plt.imshow(image_np)
plt.show()

C:\Users\USER\Desktop\Cyber\PRJ\1.jpg


error: OpenCV(4.5.5) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [4]:
#@title
input_tensor = tf.convert_to_tensor(image_np)
# The model expects a batch of images, so add an axis with `tf.newaxis`
input_tensor = input_tensor[tf.newaxis,...]
detections = detect_fn(input_tensor)

num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
              for key, value in detections.items()}
detections['num_detections'] = num_detections

# detection_classes should be ints.
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

# detections['detection_boxes'][:,0] = detections['detection_boxes'][:,0] * image_np.shape[0]
# detections['detection_boxes'][:,1] = detections['detection_boxes'][:,1] * image_np.shape[1]
# detections['detection_boxes'][:,2] = detections['detection_boxes'][:,2] * image_np.shape[0]
# detections['detection_boxes'][:,3] = detections['detection_boxes'][:,3] * image_np.shape[1]

label_id_offset = 0
image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes']+label_id_offset,
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=5,
            min_score_thresh=0.5,
            agnostic_mode=False)

plt.imshow(image_np_with_detections)
plt.show()

NameError: name 'image_np' is not defined

-------------------------------------

In [5]:
cap = cv2.VideoCapture(0)
WIDTH_WEBCAM = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
HEIGHT_WEBCAM = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))


while cap.isOpened(): 
    ret, frame = cap.read()
    frame_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(frame_np)
    # The model expects a batch of images, so add an axis with `tf.newaxis`
    input_tensor = input_tensor[tf.newaxis,...]
    detections = detect_fn(input_tensor)

    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    # detections['detection_boxes'][:,0] = detections['detection_boxes'][:,0] * image_np.shape[0]
    # detections['detection_boxes'][:,1] = detections['detection_boxes'][:,1] * image_np.shape[1]
    # detections['detection_boxes'][:,2] = detections['detection_boxes'][:,2] * image_np.shape[0]
    # detections['detection_boxes'][:,3] = detections['detection_boxes'][:,3] * image_np.shape[1]

    label_id_offset = 0
    frame_np_with_detections = frame_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                frame_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=0.5,
                agnostic_mode=False)

    cv2.imshow('object detection',  cv2.resize(frame_np_with_detections, (WIDTH_WEBCAM, HEIGHT_WEBCAM)))
    if cv2.waitKey(10) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break

-------------------------------